<a target="_blank" href="https://colab.research.google.com/github/giordamaug/WisardLib4Python/blob/main/test_follower.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Installation

In [ ]:
!git clone https://github.com/giordamaug/WisardLib4Python.git
%cd WisardLib4Python
!pip install pybind11
!pip install opencv_jupyter_ui
!python setup.py build_ext --inplace 

## Load the video

In [ ]:
import opencv_jupyter_ui as jcv2
import numpy as np
import cv2
import wisard as wnn
import time
from multiprocessing.pool import ThreadPool as Pool
import os
from functools import partial
try:
    from google.colab import output
    output.enable_custom_widget_manager()
except:
    pass

def monitor(results):
    zoom = 4
    matrix_normalized = cv2.normalize(results, None, 0, 255, cv2.NORM_MINMAX)
    matrix_uint8 = np.uint8(matrix_normalized)
    heatmap = cv2.applyColorMap(matrix_uint8, cv2.COLORMAP_HOT)
    resized_heatmap = cv2.resize(heatmap, (zoom*heatmap.shape[0], zoom*heatmap.shape[1]))
    return resized_heatmap

def draw_text(img, text,
          font=cv2.FONT_HERSHEY_PLAIN,
          pos=(0, 0),
          font_scale=3,
          font_thickness=2,
          text_color=(0, 255, 0),
          text_color_bg=(0, 0, 0)
          ):

    x, y = pos
    text_size, _ = cv2.getTextSize(text, font, font_scale, font_thickness)
    text_w, text_h = text_size
    cv2.rectangle(img, pos, (x + text_w, y + text_h), text_color_bg, -1)
    cv2.putText(img, text, (x, y + text_h + font_scale - 1), font, font_scale, text_color, font_thickness)
    return text_size

class Follower:
     def __init__(self, frame:np.ndarray, w1:int, w2:int, h1:int, h2:int, name:int, dx:int, dy:int, res:int, color=(0,0,0), minthr=5, n_bits:int=16, map:int=0):
        self.name = name
        self.w1 = w1
        self.w2 = w2
        self.h1 = h1
        self.h2 = h2
        self.dx = dx
        self.dy = dy
        self.res = res
        self.deltax = 0 
        self.deltay = 0
        self.color = color
        self.width = frame.shape[1]
        self.height = frame.shape[0]
        self.n_bits = n_bits
        self.map = map
        self.minthr = minthr
        self.w,self.h = self.w2-self.w1, self.h2-self.h1
        self.ww1, self.hh1 = max(0, self.w1-self.dx*self.res),max(0, self.h1-self.dy*self.res)
        self.ww2, self.hh2 = min(self.height, self.w2+self.dx*self.res),min(self.width, self.h2+self.dy*self.res)
        if self.w1-self.dx*self.res < 0:
            self.neww1 = self.dx*self.res - self.w1
        else:  # left outbound
            self.neww1 = 0
        if self.h1-self.dy*self.res < 0:
            self.newh1 = self.dy*self.res - self.h1
        else:  # top outbound
            self.newh1 = 0
        # crop image window including dx/dy movements
        self.crop_img = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)[self.ww1:self.ww2,self.hh1:self.hh2]
        _,self.crop_img = cv2.threshold(self.crop_img,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)

        # extend cropped image if window outbounds
        self.crop_img_ext = np.ndarray((w2-w1+2*dx*res, h2-h1+2*dy*res), dtype=self.crop_img.dtype)
        self.crop_img_ext.fill(255)
        self.crop_img_ext[self.neww1:self.neww1+self.crop_img.shape[0],self.newh1:self.newh1+self.crop_img.shape[1]] = self.crop_img
        self.crop_img_ext = np.where(self.crop_img_ext < 128, 1,0)

        # cut training image from cropped (extended) window
        self.crop_img_train = self.crop_img_ext[self.dx*self.res:self.dx*self.res+(self.w2-self.w1),self.dy*self.res:self.dy*self.res+(self.h2-self.h1)]
        self.crop_img_train_g = (self.crop_img_train * 255).astype("uint8")
        # initi dicriminator and train on image
        self._discr = wnn.WiSARD((self.w)*(self.h), n_bits=self.n_bits, map=self.map, classes = [name])
        self._discr.train(self.crop_img_train.flatten(), name)

     def response(self):
        # initialize movement responses
        result_mat = np.zeros((2 * self.dx +1, 2 * self.dy +1), dtype=float)
        # compute tuples for predition of discriminator shifte non different windows
        tuple_mat = self._discr._mk_tuple_img_multi(self.crop_img_ext, self.h, dx=self.dx, dy=self.dy, res=self.res)
        thresh = max(0, self._discr.getTcounts()[self.name] - self.minthr)
        #print(self._discr.getTcounts())
        for i in range(2 * self.dy +1):
            for j in range(2 * self.dx +1):
                result_mat[j,i] = self._discr.response_tpl(tuple_mat[i,j], threshold=thresh)[self.name]
        # get pax response and its location in response matrix
        maxidx = np.unravel_index(np.argmax(result_mat, axis=None), result_mat.shape)
        self.deltay, self.deltax = int((maxidx[0]-self.dy)*self.res), int((maxidx[1]-self.dx)*self.res)
        self.maxres = result_mat[*maxidx]
        self.monitor = monitor(result_mat)    

     def update(self, frame:np.ndarray):
        if self.deltax != 0 or self.deltay != 0:
            #print(f"MOVE[{self.name}]", self.deltax,self.deltay)
            if self.w1 + self.deltay > -1 and self.w2 + self.deltay < self.width: 
                self.w1 += self.deltay
                self.w2 += self.deltay 
            if self.h1 + self.deltax > -1 and self.h2 + self.deltax < self.height: 
                self.h1 += self.deltax
                self.h2 += self.deltax
            self.w,self.h = self.w2-self.w1, self.h2-self.h1
            self.ww1, self.hh1 = max(0, self.w1-self.dx*self.res),max(0, self.h1-self.dy*self.res)
            self.ww2, self.hh2 = min(self.height, self.w2+self.dx*self.res),min(self.width, self.h2+self.dy*self.res)
            if self.w1-self.dx*self.res < 0:
                self.neww1 = self.dx*self.res - self.w1
            else:  # left outbound
                self.neww1 = 0
            if self.h1-self.dy*self.res < 0:
                self.newh1 = self.dy*self.res - self.h1
            else:  # top outbound
                self.newh1 = 0
            self._discr.reinit()
            #self._discr.untrain(self.name)  ... not working well!

        # crop image window including dx/dy movements
        self.crop_img = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)[self.ww1:self.ww2,self.hh1:self.hh2]
        _,self.crop_img = cv2.threshold(self.crop_img,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
        self.crop_img = (255-self.crop_img)

        # extend cropped image if window outbounds
        self.crop_img_ext = np.ndarray((self.w2-self.w1+2*self.dx*self.res, self.h2-self.h1+2*self.dy*self.res), dtype=self.crop_img.dtype)
        self.crop_img_ext.fill(255)
        self.crop_img_ext[self.neww1:self.neww1+self.crop_img.shape[0],self.newh1:self.newh1+self.crop_img.shape[1]] = self.crop_img
        self.crop_img_ext = np.where(self.crop_img_ext < 128, 1,0)

        # cut training image from cropped (extended) window
        self.crop_img_train = self.crop_img_ext[self.dx*self.res:self.dx*self.res+(self.w2-self.w1),self.dy*self.res:self.dy*self.res+(self.h2-self.h1)]
        self.crop_img_train_g = (self.crop_img_train * 255).astype("uint8")
        # initi dicriminator and train on image
        self._discr.train(self.crop_img_train.flatten(), self.name)

def following(filename, windows, parallel = True, output=True, debug=False):
    cap = cv2.VideoCapture(filename)
    idxframe = 1
    nproc = len(windows)
    fps = 0
    def follower_worker(d, frame):
        d.response()
        d.update(frame)

    ret, frame = cap.read()
    if not ret:
        raise Exception("No video input")
    frame = cv2.GaussianBlur(frame,(5,5),0)
    window_name = f"{os.path.splitext(os.path.basename(filename))[0]} ({frame.shape[0]}x{frame.shape[1]})"
    followers = [Follower(frame, *win) for win in windows]
    if output: f = open('out.txt', 'w')
    while(cap.isOpened()):
        try:
            tStart=time.time()
            ret, frame = cap.read()
            if ret:
                outframe = frame.copy()
                frame = cv2.GaussianBlur(frame,(5,5),0)
                if parallel:
                    with Pool(processes=nproc) as pool:
                        pool.map(partial(follower_worker, frame=frame), followers)
                else:
                    [follower_worker(d, frame) for d in followers]
                for d in followers:
                    if debug:
                        outframe[d.w1:d.w2,d.h1:d.h2] = cv2.cvtColor(d.crop_img_train_g, cv2.COLOR_GRAY2BGR) 
                        startw = d.w2 if d.w2+d.monitor.shape[0] < frame.shape[0] else d.w1-d.monitor.shape[0]
                        outframe[startw:startw+d.monitor.shape[0],d.h1:d.h1+d.monitor.shape[1]] = d.monitor
                    frame = cv2.rectangle(frame, (d.h1-1,d.w1-1), (d.h2,d.w2), d.color, 2)
                    _,_ = draw_text(frame, f"ID:{d.name} {d.maxres:.2f}", font_scale=1, pos=(d.h1-3,d.w1-13), text_color=(255, 255, 255), text_color_bg=d.color, font_thickness=1)
                    
                if output:
                        f.write(",".join([f"[ID:{d.name}]{(d.w1+d.w2)//2},{(d.h1+d.h2)//2}" for d in followers]))
                        f.write("\n")
                cv2.putText(frame, f"fps:{fps:.0f} F:{idxframe}", (15,15), cv2.FONT_HERSHEY_PLAIN, 1, (255, 0, 0), 1)
                if debug:
                    jcv2.imshow(window_name,np.hstack((frame, outframe)))
                else:
                    jcv2.imshow(window_name,frame)
                if jcv2.waitKey(0.1)=='q':
                    break
                idxframe +=1
                fps=1.0/(time.time()-tStart)
            else:
                break
        except Exception as e:
            print(e)
            break
    if output: f.close()
    jcv2.destroyAllWindows() #optional, only needed if you don't run it in notebook
    cap.release()


In [23]:
# window (startx,endx,starty,endy), x is vertical axis: row index)
windows = [(120,170,20,100, 0, 5,5,1, (255,0, 0), 5), 
           (160,214,200,290, 1, 5,5,1, (0,100, 0), 5)]
following("data/clouds.mp4", windows,
          parallel=True, output=True, debug=True)

{0: 1}{1: 1}



[ WARN:0@848.947] global /private/var/folders/k1/30mswbxs7r1g6zwn8y4fyt500000gp/T/abs_11nitadzeg/croot/opencv-suite_1691620374638/work/modules/videoio/src/cap_gstreamer.cpp (2386) handleMessage OpenCV | GStreamer warning: your GStreamer installation is missing a required plugin
[ WARN:0@848.947] global /private/var/folders/k1/30mswbxs7r1g6zwn8y4fyt500000gp/T/abs_11nitadzeg/croot/opencv-suite_1691620374638/work/modules/videoio/src/cap_gstreamer.cpp (2402) handleMessage OpenCV | GStreamer warning: Embedded video playback halted; module uridecodebin12 reported: Your GStreamer installation is missing a plug-in.
[ WARN:0@848.948] global /private/var/folders/k1/30mswbxs7r1g6zwn8y4fyt500000gp/T/abs_11nitadzeg/croot/opencv-suite_1691620374638/work/modules/videoio/src/cap_gstreamer.cpp (1356) open OpenCV | GStreamer warning: unable to start pipeline
[ WARN:0@848.948] global /private/var/folders/k1/30mswbxs7r1g6zwn8y4fyt500000gp/T/abs_11nitadzeg/croot/opencv-suite_1691620374638/work/modules/vide

{0: 2}{1: 2}

{1: 3}
{0: 3}
{0: 4}
{1: 4}
{0: 5}
{1: 5}
{0: 6}
{1: 6}
{0: 7}
{1: 7}
{0: 8}
{1: 8}
{0: 9}
{1: 9}
{0: 10}
{1: 10}
{0: 11}
{1: 11}
{0: 12}
{1: 12}
{0: 13}
{1: 13}
{0: 14}{1: 14}

{0: 15}
{1: 15}
{0: 20}{1: 20}

{0: 21}
{1: 21}
{0: 22}
{1: 22}
{0: 23}
{1: 23}
{0: 24}
{1: 24}
{0: 25}
{1: 25}
{0: 26}
{1: 26}
{0: 27}
{1: 27}
{0: 28}
{1: 28}
{0: 29}
{1: 29}
{0: 30}
{1: 30}
{0: 31}
{1: 31}
{0: 32}
{1: 32}
{0: 33}
{1: 33}
{0: 34}
{1: 34}
{0: 35}
{1: 35}
{0: 36}
{1: 36}
{0: 37}
{1: 37}
{0: 38}
{1: 38}
{1: 39}
{0: 39}
{1: 40}
{0: 40}
{1: 41}{0: 41}

{1: 42}
{0: 42}
{0: 43}
{1: 43}
{0: 44}
{1: 44}
{0: 45}
{1: 45}
{0: 46}{1: 46}

{0: 47}
{1: 47}
{0: 48}
{1: 48}
{0: 49}
{1: 49}
{0: 50}
{1: 50}
{0: 51}
{1: 51}
{0: 52}
{1: 52}
{0: 53}
{1: 53}
{1: 54}
{0: 54}
{0: 55}
{1: 55}
{0: 56}
{1: 56}
{0: 57}
{1: 57}
{0: 58}
{1: 58}
{0: 59}
{1: 59}
{0: 60}{1: 60}

{0: 61}
{1: 61}
{0: 62}
{1: 62}
{0: 63}
{1: 63}
{0: 64}
{1: 64}
{0: 65}
{1: 65}
{0: 66}
{1: 66}
{0: 67}
{1: 67}
{0: 68}
{1: 68}
{0: 69}
